# 1. check valid json format

In [3]:
import os,json

f = 'test.json'
with open(f, 'r') as fr:
    d = json.load(fr)

In [6]:
len(d), [k for k in d[0]]

(120, ['image', 'objects'])

In [7]:
d[0]['image']

{'checksum': 'eea3bfd6a929bcb06f9786667cd3fbb2',
 'pathname': '/images/41be1bd3-0d31-4881-bf1f-3ccdfa21ff12.jpg',
 'shape': {'r': 1383, 'c': 1944, 'channels': 3}}

In [8]:
len(d[0]['objects'])

67

In [9]:
d[0]['objects'][0]

{'bounding_box': {'minimum': {'r': 576, 'c': 1744},
  'maximum': {'r': 708, 'c': 1883}},
 'category': 'red blood cell'}

In [14]:
import cv2

image = cv2.imread('微信图片_20230813195112.jpg')
image.shape

(1383, 1944, 3)

In [26]:
# check categories

import os,json

set_category = set([])

f = 'test.json'
with open(f, 'r') as fr:
    d = json.load(fr)
for dd in d:
    for ddd in dd['objects']:
        c = ddd['category']
        set_category.add(c)
        

f = 'training.json'
with open(f, 'r') as fr:
    d = json.load(fr)
for dd in d:
    for ddd in dd['objects']:
        c = ddd['category']
        set_category.add(c)

        
        
set_category


{'difficult',
 'gametocyte',
 'leukocyte',
 'red blood cell',
 'ring',
 'schizont',
 'trophozoite'}

# 2. all manually small json files to one big json

In [11]:
import hashlib
 
filename = 'vivax/img/1703121298-0001-G_R.jpg'
with open(filename,"rb") as f:
    bytes = f.read() # read file as bytes
    readable_hash = hashlib.md5(bytes).hexdigest();
    print(readable_hash)


6e9ed25930b509a32bf95b09f47041a7


In [15]:
import hashlib
import cv2

 
def get_md5(filename):
    result = None
    try:
        with open(filename,"rb") as f:
            bytess = f.read() # read file as bytes
            readable_hash = hashlib.md5(bytess).hexdigest();
            result = readable_hash
    except:
        pass
    return result


def get_shape(filename):
    image = cv2.imread(filename)
    r,c,channels = image.shape
    return r,c,channels


m = get_md5(  'vivax/img/1703121298-0001-G_R.jpg'  )
r,c,channels = get_shape('微信图片_20230813195112.jpg')
print(m)
print(r,c,channels)

6e9ed25930b509a32bf95b09f47041a7
1383 1944 3


In [39]:
import os
import json


manual_name_to_category = {
    'RBC':'red blood cell',
    'Gametocyte':'gametocyte',
    'Ring':'ring',
    'Schizont':'schizont',
    'Trophozoite':'trophozoite'
}


m_json_list = []
f1 = 'vivax/img/'
for f2 in os.listdir(f1):
    f3 = os.path.join(f1,f2)
    if f3.endswith('.json'):
        m_json_list.append(f3)
print('get small json files : {0}'.format(len( m_json_list )))

result = []

for f in m_json_list:
    with open(f, 'r') as fr:
        d = json.load(fr)
        tmp = {}
        tmp['image'] = {}
        img_f = f.replace('.json', '.jpg')
        tmp['image']['checksum'] = get_md5(img_f)
        tmp['image']['pathname'] = img_f
        r,c,channels = get_shape(img_f)
        tmp['image']['shape'] = {'r': r, 'c': c, 'channels': channels}
        
        tmp['objects'] = []
        for dd in d['shapes']:
            label = dd['label']
            r1,c1 = dd['points'][0]
            r2,c2 = dd['points'][1]
            r1 = int(r1)
            c1 = int(c1)
            r2 = int(r2)
            c2 = int(c2)
            tmp2 = {}
            tmp2['bounding_box'] = {}
            tmp2['bounding_box']['minimum'] = {'r': r1, 'c': c1}
            tmp2['bounding_box']['maximum'] = {'r': r2, 'c': c2}
            tmp2['category'] = manual_name_to_category[label]
            tmp['objects'].append(tmp2)
        
    result.append(tmp)

get small json files : 40


In [40]:
len(result)

40

In [41]:
import json

# Serializing json
json_object = json.dumps(result)

# Writing to sample.json
with open("test_vivax.json", "w") as outfile:
    outfile.write(json_object)

In [46]:
for d in result:
    objects = d['objects']
    for obj in objects:
        if 'bounding_box' not in obj:
            print('error', obj)